# Building Name Extraction

In [28]:
# imports 
import data_preprocessor
from config import *

abbreviations = data_preprocessor.load_json("abbreviations.json")

In [29]:
df = data_preprocessor.load_corpus(fname, pandas = True, header = True)
# df = df.drop(columns=['Title', 'Created', 'Close Time', 'Queue'], axis=1) 
df = df.drop(columns=['Building Name'], axis=1) 

df.head()

,Ticket#,Address
0,1,"House # B6, Block-B Floor Aftab Sultan residen..."
1,2,"Apartment/Suite# B-3 , Building Block B, Aftab..."
2,3,"Apartment/Suite# B-1 1st Floor, Building Block..."
3,4,House # Aftab Sultan Resedention complex Appt ...
4,5,House # St 20 fL B2 Aftab sultan near postoffi...


In [30]:
df_normalized = data_preprocessor.load_corpus(fname_normalized, pandas = True, header = True)
df_normalized = df_normalized.fillna('None') # -> if from file

# Merging the DataFrames based on 'Ticket #' to append the 'Address' column
df_normalized = df_normalized.merge(df, left_on='Ticket #', right_on='Ticket#', how='left').drop('Ticket#', axis=1)

df_normalized.head()

,Ticket #,Type,House #,Apartment #,Building #,Building Name,Street,Road,Area & Sub Area,Neighbourhood,City,Address
0,1,house,house # b6,block - b floor aftab sultan residencial complex,None,None,None,shahrah e ghalib,block 2,clifton,karachi,"House # B6, Block-B Floor Aftab Sultan residen..."
1,2,apartment,None,apartment / suite # b - 3,building block b,aftab sultan residential complex,None,khayaban - e - bedi,block 2,clifton,karachi,"Apartment/Suite# B-3 , Building Block B, Aftab..."
2,3,apartment,None,apartment / suite # b - 1 1st floor,building block b,aftab sultan residential complex,None,khayaban - e - bedi,block 2,clifton,karachi,"Apartment/Suite# B-1 1st Floor, Building Block..."
3,4,apartment,None,house # aftab sultan resedention complex apart...,None,None,None,khayaban - e - bedi,block 2,clifton,karachi,House # Aftab Sultan Resedention complex Appt ...
4,5,house,None,None,None,None,house # street 20 floor b2 aftab sultan near p...,khayaban - e - bedi,block 2,clifton,karachi,House # St 20 fL B2 Aftab sultan near postoffi...


In [31]:
lst_addresses = df_normalized[df_normalized['Building Name'] == 'None']['Address'].tolist()
lst_building_names = list(set(df_normalized[(df_normalized['Building Name'] != 'None') & (df_normalized['Type'] == 'apartment')]['Building Name']))

print(lst_addresses)
print(lst_building_names)


['House # B6, Block-B Floor Aftab Sultan residencial Complex, Shahra e Ghalib, Block 2, Clifton, Karachi', 'House # Aftab Sultan Resedention complex Appt A/3 2nd Fl, Khayaban-e-Bedi, Block 2, Clifton, Karachi', 'House # St 20 fL B2 Aftab sultan near postoffice, Khayaban-e-Bedi, Block 2, Clifton, Karachi', 'House # Aftab Sultan Complex Appt A6, Khayaban-e-Bedi, Block 2, Clifton, Karachi', 'House # ST-20 Flat B-4 2nd Floor Aftab Sultan Residential, Khayaban-e-Bedi, Block 2, Clifton, Karachi', 'House # C-7 3rd Floor Al Habib Castle, 3rd Avenue, Block 5, Clifton, Karachi', 'House # Blessed Home, Flat E-5, McNeil Road, Civil Lines, Clifton, Karachi', 'House # 37 Flat F 2 1st Floor Blessed Homes, Mc Neil Road, Civil Lines, Clifton, Karachi', 'House # Blessed Homes Appt # E-6 5TH Floor, McNeil Road, Civil Lines, Clifton, Karachi', 'House # 37-A Flat D-4 3rd Floor Blessed Homes, McNeil Road, Civil Lines, Clifton, Karachi', 'House # 37 A Apartment No C 7 F Blessed Homes, Adam Road, Civil Lines,

In [13]:
# takes address from its source --> source can be df or file
# takes building name from its source --> source can be df or file
# appends address to respective normalized from ticket # as reference
# takes addresses to which Building Name is None
# takes building names which is not None and type is apartment
# refinement in building names; remove building all blocks, building block, building a,b,c,d..., block 1,2,3, block a,b,c,..., 
def preparing_extraction(address_df, normalized_df):
    normalized_df = normalized_df.merge(address_df, left_on='Ticket #', right_on='Ticket#', how='left').drop('Ticket#', axis=1)
    
    lst_addresses = normalized_df[normalized_df['Building Name'] == 'None']['Address'].tolist()
    lst_building_names = list(set(normalized_df[(normalized_df['Building Name'] != 'None') & (normalized_df['Type'] == 'apartment')]['Building Name']))

    cleaned_building_names = refine_building_names(lst_building_names)
    
    return lst_addresses, cleaned_building_names

# to be called in preparing_extraction(), refines building names by removing outliers in data
def refine_building_names(building_list):
    refined_list = [name for name in building_list if not re.match(r'^(building( all blocks| block)?|block)( [a-zA-Z0-9]+)?$', name, re.I)]
    refined_list.sort(key=lambda x: (len(x) <= 4, x))
    return refined_list


# the 2 building name extraction from chatgpt --> test both then use best one. 
    # from the refined building name corpus, check for building names in address
# return address and building name (correct) if present, otherwise return none
def layer2checks__extract_building_names():
    pass


# identify field in which the address is wrongly placed
# change spelling in address if incorrect
# remove/delete building name from that field and place in building name column 
# if None not in building name:
    # check based on probability identifier mechanim
    # either place in buildng number colum or area/subarea column or other relevant place
# if No building name found then go to city building corpus and check in that
# correct other fields if they are misplaced as well.
def correction():
    pass


# when all previous done, have city building corpus ready
# check in it it building is present, if not then place new buildings in the corpus
# that way we can have a city wise building corpus/directory which is self learning and self updating
# the more runs and more it learns and saves and better results acheived
def updating_building_corpus():
    pass 

# all if building is found

address, building_name = 'House # B6, Block-B Floor, Shahra e Ghalib, Block 2, Clifton, Karachi', 'Aftab Sultan residencial Complex'

print(address)
print(building_name)

'House # Aftab Sultan Resedention complex Appt A/3 2nd Fl, Khayaban-e-Bedi, Block 2, Clifton, Karachi'


House # B6, Block-B Floor, Shahra e Ghalib, Block 2, Clifton, Karachi
Aftab Sultan residencial Complex


In [32]:
import re

def refine_building_names(building_list):
    refined_list = [name for name in building_list if not re.match(r'^(building( all blocks| block)?|block)( [a-zA-Z0-9]+)?$', name, re.I)]
    refined_list.sort(key=lambda x: (len(x) <= 4, x))
    return refined_list

# Example list
building_list = [
    "Building Block 1",
    "Building Block 2",
    "Building Block A",
    "Building Block B",
    "Building all blocks",
    "Building Block",
    "Building A",
    "Building B",
    "Building 1",
    "Building",
    'block 30',
    'block t',
    'block 8',
    'building all blocks',
    'building m',
    'building block t',
    'building d',
    "Mansion Z",
    "Estate X",
    "Flat"
]

refined_buildings = refine_building_names(building_list)
print(refined_buildings)


['Estate X', 'Mansion Z', 'Flat']
